# Simpsons Detector
## Creating dataset

The following Python notebook refers the [simpsons detector post](http://zachmoshe.com).

This notebook shows how to generate a tagged video given a trained model.

Some classes are from my `general` and `simpsons` modules. They are available in [my github repo](https://github.com/zachmoshe/datalab)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import keras
import sklearn
import pickle

import sys
import logging 
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger("py.warnings").setLevel(logging.ERROR)

%matplotlib inline

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 

Using TensorFlow backend.


In [2]:
import general

In [3]:
import simpsons

In [4]:
BASE_PATH = "/Users/zach/proj/datalab-data/simpsons/"

In [5]:
config = dict(
    training_img_shape=(202,360),     # 1/4 of HD image
    test_img_shape=(202,360),         # ...
)

# Load model and thresholds

In [7]:
import pickle
tmp = pickle.load(open(BASE_PATH+"model_and_thresholds.pickle", "rb"))
model, preprocess_pipeline, thresholds = tmp["model"], tmp["preprocess_pipeline"], tmp["thresholds"]

# Load datasets

In [6]:
import gzip
import pickle

with gzip.GzipFile(BASE_PATH+"dataset.pickle.zip", "r") as f:
    data = pickle.load(f)

print("train images by char: ", data.train.y.sum(axis=0))
print("train_dev images by char: ", data.train_dev.y.sum(axis=0))
print("dev  images by char: ", data.dev.y.sum(axis=0))
print("test images by char: ", data.test.y.sum(axis=0))
data.show()

train images by char:  [ 91.  79.  64.  37.]
train_dev images by char:  [ 30.  26.  22.  12.]
dev  images by char:  [  74.  101.   66.   65.]
test images by char:  [  73.  102.   64.   64.]
train
  - X: (474, 200, 200, 3)
  - y: (474, 4)
train_dev
  - X: (158, 200, 200, 3)
  - y: (158, 4)
dev
  - X: (351, 202, 360, 3)
  - y: (351, 4)
test
  - X: (351, 202, 360, 3)
  - y: (351, 4)


# Generate a Tagged Video

In [6]:
tagger = simpsons.video.SimpsonsVideoTagger(model, thresholds, preprocess_pipeline=preprocess_pipeline)

In [7]:
clip = tagger.tag(BASE_PATH+"preprocessing/test/full-episodes/<<episode filename>>", 
                  extractor_params=dict(frames_per_second=4))

In [8]:
clip.write_videofile(BASE_PATH+"video/video_tagged.mp4", fps=24)